In [1]:
# train a simple classification model to predict if a repo is developer tooling or not

## xgboost

In [1]:
# run imports
from sqlalchemy import text
import sqlalchemy
import psycopg2
import os
import pandas as pd
from sqlalchemy import create_engine, text
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os

In [2]:
# --- Database Connection and Feature Retrieval ---
cloud_sql_user = os.getenv("cloud_sql_user")
cloud_sql_password = os.getenv("cloud_sql_password")
cloud_sql_postgres_host = os.getenv("cloud_sql_postgres_host")
cloud_sql_postgres_db = os.getenv("cloud_sql_postgres_db")

# Construct the connection string
conn_str = (
    f"postgresql+psycopg2://"
    f"{cloud_sql_user}:{cloud_sql_password}@"
    f"{cloud_sql_postgres_host}/{cloud_sql_postgres_db}"
)

# Create the SQLAlchemy engine
try:
    cloud_sql_engine = create_engine(conn_str)
except Exception as e:
    raise Exception(f"Error creating database engine: {e}")

In [3]:
# Fetch the feature dataset from the database
try:
    with cloud_sql_engine.connect() as conn:
        query = text(
            """
            SELECT 
                repo,
                has_readme,
                is_collection_of_learnings,
                has_app_application,
                is_awesome_curated,
                has_benchmark,
                is_block_explorer,
                is_boilerplate_scaffold_template,
                is_bootcamp,
                is_bot,
                has_bounty_program,
                has_brand_icon_logo,
                is_cli_tool,
                is_library,
                is_course,
                is_demo,
                has_docs,
                is_education_related,
                is_eip_erc,
                has_examples,
                is_feature_description,
                is_starter_project,
                is_guide,
                is_hackathon_project,
                is_hello_world,
                uses_json_rpc,
                is_interview_related,
                is_learning_material,
                is_mcp_server,
                is_plugin,
                is_sample_project,
                is_sdk,
                is_security_related,
                has_tests_testing,
                has_tips,
                is_tooling,
                is_tutorial,
                is_whitepaper,
                is_workshop,
                is_wrapper,
                name_is_example,
                name_is_hello_world,
                name_is_whitepaper,
                name_is_tutorial,
                name_is_boilerplate,
                name_is_scaffold,
                name_is_template,
                name_is_kit,
                name_is_starter,
                name_is_getting_started,
                name_is_quickstart,
                name_is_guide,
                name_is_hackathon,
                name_is_bootcamp,
                name_is_course,
                name_is_workshop,
                name_is_interview

            FROM clean.project_repos_features
            """
        )
        repo_features_df = pd.read_sql(query, conn)
except Exception as e:
    raise Exception(f"Error fetching data from the database: {e}")

In [18]:
# --- Labeled Data Retrieval ---
gsheet_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTbKlg5CJYNO3d2lrRbWWUy-71sr-oBApAJxCx2xmV931Y8CDrJ46SwVCUEoOT90LZsPpALVS_QixkE/pub?gid=1690796422&single=true&output=csv'
try:
    educational_df = pd.read_csv(gsheet_url)
except Exception as e:
    raise Exception(f"Error reading data from Google Sheets: {e}")

In [5]:
# --- Data Preparation ---
# Merge the feature data with the labeled data
merged_df = pd.merge(repo_features_df, educational_df, on='repo')

In [6]:
# --- Data Preparation ---
# Merge the feature data with the labeled data
merged_df = merged_df.dropna(subset=['is_educational'])

In [7]:
# print info about the merged dataframe
print(f"Merged dataframe info: {merged_df.info()}")

# print the first 5 rows of the merged dataframe
print(f"Merged dataframe first 5 rows: {merged_df.head()}")

# print the number of rows where is_dev_tooling is 1
print(f"Number of rows where is_educational is 1: {merged_df[merged_df['is_educational'] == 1].shape[0]}")

# print the number of rows where is_dev_tooling is 0
print(f"Number of rows where is_educational is 0: {merged_df[merged_df['is_educational'] == 0].shape[0]}")

# print the number of rows where is_dev_tooling is null
print(f"Number of rows where is_educational is null: {merged_df[merged_df['is_educational'].isnull()].shape[0]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 59 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   repo                              626 non-null    object
 1   has_readme                        626 non-null    bool  
 2   is_collection_of_learnings        626 non-null    bool  
 3   has_app_application               626 non-null    bool  
 4   is_awesome_curated                626 non-null    bool  
 5   has_benchmark                     626 non-null    bool  
 6   is_block_explorer                 626 non-null    bool  
 7   is_boilerplate_scaffold_template  626 non-null    bool  
 8   is_bootcamp                       626 non-null    bool  
 9   is_bot                            626 non-null    bool  
 10  has_bounty_program                626 non-null    bool  
 11  has_brand_icon_logo               626 non-null    bool  
 12  is_cli_tool           

In [8]:
# Separate features (X) and the target variable (y)
feature_columns = [
    'has_readme', # if false, then not dev tooling
    'is_collection_of_learnings', # if true, then not dev tooling
    'has_app_application', # if true, then not dev tooling
    'is_awesome_curated', # if true, then not dev tooling
    'has_benchmark', # if true, then not dev tooling
    'is_block_explorer', # if true, then not dev tooling
    'is_boilerplate_scaffold_template', # if true, then not dev tooling
    'is_bootcamp', # if true, then not dev tooling
    'is_bot', # if true, then not dev tooling
    'has_bounty_program', # if true, then not dev tooling
    'has_brand_icon_logo', # if true, then not dev tooling
    'is_cli_tool', # if true, then dev tooling
    'is_library', # if true, then dev tooling
    'is_course', # if true, then not dev tooling
    'is_demo', # if true, then not dev tooling
    'has_docs', # if true, then not dev tooling
    'is_education_related', # if true, then not dev tooling
    'is_eip_erc', # if true, then not dev tooling
    'has_examples', # if true, then not dev tooling
    'is_feature_description', # if true, then not dev tooling
    'is_starter_project', # if true, then not dev tooling
    'is_guide', # if true, then not dev tooling
    'is_hackathon_project', # if true, then not dev tooling
    'is_hello_world', # if true, then not dev tooling
    'uses_json_rpc', # if true, then dev tooling
    'is_interview_related', # if true, then not dev tooling
    'is_learning_material', # if true, then not dev tooling
    'is_mcp_server', # if true, then not dev tooling
    'is_plugin', # if true, then dev tooling
    'is_sample_project', # if true, then not dev tooling
    'is_sdk', # if true, then dev tooling
    'is_security_related', # if true, then not dev tooling
    'has_tests_testing', # if true, then not dev tooling
    'has_tips', # if true, then not dev tooling
    'is_tooling', # if true, then dev tooling
    'is_tutorial', # if true, then not dev tooling
    'is_whitepaper', # if true, then not dev tooling
    'is_workshop', # if true, then not dev tooling
    'is_wrapper', # if true, then dev tooling
    'name_is_example', # if true, then not dev tooling
    'name_is_hello_world', # if true, then not dev tooling
    'name_is_whitepaper', # if true, then not dev tooling
    'name_is_tutorial', # if true, then not dev tooling
    'name_is_boilerplate', # if true, then not dev tooling
    'name_is_scaffold', # if true, then not dev tooling
    'name_is_template', # if true, then not dev tooling
    'name_is_kit', # if true, then dev tooling
    'name_is_starter', # if true, then not dev tooling
    'name_is_getting_started', # if true, then not dev tooling
    'name_is_quickstart', # if true, then not dev tooling
    'name_is_guide', # if true, then not dev tooling
    'name_is_hackathon', # if true, then not dev tooling
    'name_is_bootcamp', # if true, then not dev tooling
    'name_is_course', # if true, then not dev tooling
    'name_is_workshop', # if true, then not dev tooling
    'name_is_interview' # if true, then not dev tooling
] 

X = merged_df[feature_columns]

# Ensure all feature data is numeric (booleans will be treated as 0s and 1s)
X = X.astype(float)

y = merged_df['is_educational']

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
## ----------------------------------------------------- Model Training ------------------------------------------------- ##

In [11]:
# set the n_estimators param
n_estimators = 500

In [12]:
# --- Model Training ---
# Initialize and train an XGBoost model
# For imbalanced datasets, scale_pos_weight is the equivalent of class_weight='balanced'
# It's calculated as: count(negative_class) / count(positive_class)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

model = xgb.XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight, n_estimators=n_estimators)
model.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [13]:
## ----------------------------------------------------- Model Evaluation ------------------------------------------------- ##

In [14]:
# Make predictions on the test set
y_pred = model.predict(X_test)

In [15]:
# Calculate and print the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}\n")

# Print a detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.8175

Classification Report:
              precision    recall  f1-score   support

       False       0.85      0.94      0.89       103
        True       0.50      0.26      0.34        23

    accuracy                           0.82       126
   macro avg       0.68      0.60      0.62       126
weighted avg       0.79      0.82      0.79       126



In [16]:
# --- Confusion Matrix with Labels ---
# Get the confusion matrix
cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print("                 Predicted")
print("                 False    True")
print("Actual False    {:<8} {:<8}".format(cm[0][0], cm[0][1]))
print("       True     {:<8} {:<8}".format(cm[1][0], cm[1][1]))
print("\n")

# Explanation of the terms
tn, fp, fn, tp = cm.ravel()
print(f"True Negatives (TN): {tn} - Correctly predicted not educational")
print(f"False Positives (FP): {fp} - Incorrectly predicted as educational")
print(f"False Negatives (FN): {fn} - Incorrectly predicted as not educational (missed)")
print(f"True Positives (TP): {tp} - Correctly predicted as educational")

Confusion Matrix:
                 Predicted
                 False    True
Actual False    97       6       
       True     17       6       


True Negatives (TN): 97 - Correctly predicted not educational
False Positives (FP): 6 - Incorrectly predicted as educational
False Negatives (FN): 17 - Incorrectly predicted as not educational (missed)
True Positives (TP): 6 - Correctly predicted as educational


In [17]:
# Get importance scores
importance = model.feature_importances_

# Get the column names from training data
feature_names = X_train.columns

# Create a pandas Series to pair feature names with their importance scores
feat_importances = pd.Series(importance, index=feature_names)

# Sort the Series in descending order (most important features first)
sorted_importances = feat_importances.sort_values(ascending=False)

# 4. Print the sorted list
print("--- Feature Importances (Sorted) ---")
print(sorted_importances)

--- Feature Importances (Sorted) ---
is_security_related                 0.120503
is_learning_material                0.069448
is_eip_erc                          0.067022
is_feature_description              0.063353
name_is_whitepaper                  0.051241
name_is_quickstart                  0.047425
name_is_interview                   0.046722
name_is_example                     0.045683
is_tutorial                         0.044686
name_is_hackathon                   0.043034
is_cli_tool                         0.040778
is_library                          0.040075
is_whitepaper                       0.039895
is_education_related                0.038993
name_is_boilerplate                 0.038206
is_awesome_curated                  0.034445
name_is_template                    0.031558
has_bounty_program                  0.024292
is_demo                             0.015771
has_benchmark                       0.014780
uses_json_rpc                       0.010546
is_sample_project 